In [1]:
import os, sys
from captcha.image import ImageCaptcha
import random
import string
import numpy as np
import torch
from torch import nn
import torchvision
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader,TensorDataset
from PIL import Image

In [2]:
def calculat_acc(output, target):
    output, target = output.view(-1,10,4), target.view(-1, 4)
#     output = nn.functional.softmax(output, dim=1)
    output = torch.argmax(output, dim=1)
    output = output.float()
    
#     target = torch.argmax(target, dim=1)
    output, target = output.view(-1, 4), target.view(-1, 4)
    correct_list = []
    for i, j in zip(target, output):
        if torch.equal(i, j):
            correct_list.append(1)
        else:
            correct_list.append(0)
    acc = sum(correct_list) / len(correct_list)
    return acc

In [3]:
#定义验证码大小，n_class,
height,width,batch_size,n_class = 50,100,32,10
#生成随机数
def get_code():
    raw = string.digits
#     + string.ascii_lowercase #小写
#     + string.ascii_uppercase#大写

    random_code = ''.join(random.sample(raw,4))
    return raw,random_code

#生成验证码
def generator(height,width,batch_size,n_class):
    X = np.zeros((batch_size,height,width,3))
    y = np.zeros((batch_size,4))
    
    #验证码生成器
    generator = ImageCaptcha(height=height,width=width)
    for i in range(batch_size):
        raw ,random_code = get_code()
        img = np.array((generator.generate_image(random_code)),dtype=np.float32)
        X[i] = img/255.0

        for step,ch in enumerate(random_code):
            y[i][step] = int(ch)
    return X,y

In [6]:
train_batch_size = 1000
test_batch_size = 128
train_data,train_label = generator(height,width,train_batch_size,n_class)
train_data = np.transpose(train_data,(0,3,1,2))
train_data = torch.from_numpy(train_data)
train_label = torch.Tensor(train_label)
train_datas = TensorDataset(train_data,train_label)

test_data,test_label = generator(height,width,test_batch_size,n_class)
test_data = np.transpose(test_data,(0,3,1,2))
test_data = torch.from_numpy(test_data)
test_label = torch.Tensor(test_label)
test_datas = TensorDataset(test_data,test_label)

In [7]:
batch_size = 32
train_data_loader = DataLoader(train_datas, batch_size=batch_size,num_workers=0, shuffle=True, drop_last=True)

test_data_loader = DataLoader(test_datas, batch_size=batch_size,num_workers=0, shuffle=True, drop_last=True)

In [ ]:
for i,j in train_data_loader:
    print(j.shape)
#     j = j.view(-1,4,4)
#     print(j.view(-1,4,4).shape)
#     print(np.argmax(j,axis=1))

In [8]:
class CNN(nn.Module):
    def __init__(self, num_class=10, num_char=4):
        super(CNN, self).__init__()
        self.num_class = num_class
        self.num_char = num_char
        self.conv = nn.Sequential(
                nn.Conv2d(3, 16, 3,stride=(1,1), padding=(2,2)),
                nn.MaxPool2d(2, 2),
                nn.ReLU(),
                nn.Conv2d(16, 32, 3,stride=(1,2), padding=(1,1)),
                nn.MaxPool2d(2, 2),
                nn.ReLU(),
                nn.Conv2d(32, 64, 3, padding=(1,1)),
                nn.MaxPool2d(2, 2),
                nn.Conv2d(64, 10, 3,stride=(2,2)),
                )
        self.act = nn.LogSoftmax(dim=1)
 
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1,10,4)#torch.Size([128, 10, 4])
#         print(x.shape)
        x = self.act(x)
        return x

In [14]:
cnn = CNN()
if torch.cuda.is_available():
    cnn.cuda()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
criterion = nn.NLLLoss(reduction='mean')

for epoch in range(80):
    cnn.train()
    loss_history = []
    acc_history = []
    for img, target in train_data_loader:
        output = cnn(img.float())
        loss = criterion(output, target.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = calculat_acc(output, target)
        acc_history.append(float(acc))
        loss_history.append(float(loss))
    print('train_loss: {:.4}|train_acc: {:.4}'.format(
            torch.mean(torch.Tensor(loss_history)),
            torch.mean(torch.Tensor(acc_history))))

    loss_history = []
    acc_history = []
#     cnn.eval()
    for img, target in test_data_loader:
        output = cnn(img.float())#torch.Size([128, 10, 4])
        loss = criterion(output, target.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = calculat_acc(output, target)
        acc_history.append(float(acc))
        loss_history.append(float(loss))
        
    print('test_loss: {:.4}|test_acc: {:.4}'.format(
            torch.mean(torch.Tensor(loss_history)),
            torch.mean(torch.Tensor(acc_history))
            ))

train_loss: 2.305|train_acc: 0.0
test_loss: 2.303|test_acc: 0.0
train_loss: 2.302|train_acc: 0.0
test_loss: 2.301|test_acc: 0.0
train_loss: 2.3|train_acc: 0.0
test_loss: 2.295|test_acc: 0.0
train_loss: 2.294|train_acc: 0.0
test_loss: 2.279|test_acc: 0.0
train_loss: 2.265|train_acc: 0.001008
test_loss: 2.229|test_acc: 0.0
train_loss: 2.205|train_acc: 0.001008
test_loss: 2.172|test_acc: 0.0
train_loss: 2.142|train_acc: 0.002016
test_loss: 2.108|test_acc: 0.0
train_loss: 2.073|train_acc: 0.00504
test_loss: 2.037|test_acc: 0.007812
train_loss: 1.993|train_acc: 0.01411
test_loss: 1.963|test_acc: 0.007812
train_loss: 1.908|train_acc: 0.01109
test_loss: 1.87|test_acc: 0.01562
train_loss: 1.816|train_acc: 0.01714
test_loss: 1.794|test_acc: 0.02344
train_loss: 1.741|train_acc: 0.02319
test_loss: 1.739|test_acc: 0.03125
train_loss: 1.66|train_acc: 0.03629
test_loss: 1.633|test_acc: 0.0625
train_loss: 1.586|train_acc: 0.03931
test_loss: 1.574|test_acc: 0.05469
train_loss: 1.516|train_acc: 0.05948

In [ ]:
torch.save(cnn.state_dict(), 'params.pkl')

In [31]:
from torchvision.transforms import Compose, ToTensor
from torchvision import transforms
transforms = Compose([
                      ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

img='C:\\Users\\yu guo long\\python_project\\验证码识别\\data\\train\\7593.jpg'
img=Image.open(img)
img = np.array(img)
img = img/255.0
img=transforms(img)
img.unsqueeze_(dim=0)


cnn.eval()
out=cnn(img.float())
# outnp=out.data[0]
out = torch.argmax(out, dim=1)
# print(out.shape)
print(out)

tensor([[4, 5, 8, 4]])


In [24]:
for img,target in test_data_loader:
#     print(target[1].shape)
    cnn.eval()
    img = img[1]
    target = target[1]
    img = img.unsqueeze_(dim=0)

output = cnn(img.float())
out = torch.argmax(output, dim=1)
print(out)
print(target)
#     loss = criterion(output, target.long())
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     acc = calculat_acc(output, target)
#     acc_history.append(float(acc))
#     loss_history.append(float(loss))
# print('train_loss: {:.4}|train_acc: {:.4}'.format(
#         torch.mean(torch.Tensor(loss_history)),
#         torch.mean(torch.Tensor(acc_history))))

tensor([[4, 1, 5, 8]])
tensor([4., 1., 5., 8.])
